In [1]:
from guidance import models, gen, user, assistant, system, silent

from autogen import config_list_from_json

llm_config = config_list_from_json("OAI_CONFIG_LIST")[0]  # use the first config
gpt = models.OpenAI("gpt-4", api_key=llm_config.get("api_key"))

with user():
    lm = gpt + "Write a hello world program in Python using proper code blocks. Only return code."
with assistant():
    lm += gen()
with user():
    lm += "Just the code blocks containing the valid code is"
with assistant():
    lm += "```python" + gen()

In [2]:
from guidance import models, gen, user, assistant, select

from autogen import config_list_from_json

llm_config = config_list_from_json("OAI_CONFIG_LIST")[0]  # use the first config
gpt = models.OpenAI("gpt-4", api_key=llm_config.get("api_key"))

with user():
    lm = gpt + "Create a json that labels the following text as funny, not funny and outputs and explanation for the classification. The text is what did the fish say when it bumped into a wall? Dam!"
with assistant():
    lm += gen()
with user():
    lm += "The label is"
with assistant():
    lm += select(['funny', 'not-funny'])
with user():
    lm += "The explantion is"
with assistant():
    lm += gen()
with user():
    lm += """Output a valid json object containing two keys label and explanation."""
with assistant():
    lm += "{" + gen()


In [3]:
from guidance import models, gen, user, assistant, select

from autogen import config_list_from_json

llm_config = config_list_from_json("OAI_CONFIG_LIST")[0]  # use the first config
gpt = models.OpenAI("gpt-4", api_key=llm_config.get("api_key"))

task = "Plot a chart of NVDA and TESLA stock price change YTD."

with user():
    lm = gpt + f"Write code to solve the following task: {task}"
with assistant():
    lm += gen()
with user():
    lm += "Does the response above contain code blocks or not?"
with assistant():
    lm += select(["yes", "no"])
with user():
    lm += "Resond with a single blocks containing the valid code. Valid code blocks start with ```"
with assistant():
    lm += "```" + gen()


In [8]:


from autogen import AssistantAgent, UserProxyAgent, Agent

def generate_structured_response(recipient, messages, sender, config):
    gpt = models.OpenAI("gpt-4", api_key=llm_config.get("api_key"), echo=False)
    
    # populate the recipient with the messages from the history
    with system():
        lm = gpt + recipient.system_message
    
    for message in messages:
        if message.get("role") == "user":
            with user():
                lm += message.get("content")
        else:
            with assistant():
                lm += message.get("content")

    # generate a new response and store it
    with assistant():
        lm += gen(name="initial_response")
    # ask the agent to reflect on the nature of the response and store it
    with user():
        lm += "Does the very last response from you contain code? Respond with yes or no."
    with assistant():
        lm += gen(name="contains_code")
    # if the response contains code, ask the agent to generate a proper code block
    if "yes" in lm["contains_code"].lower():
        with user():
            lm += "Respond with a single blocks containing the valid code. Valid code blocks start with ```"
        with assistant():
            lm += "```" + gen(name="code")
            response = "```" + lm["code"]
    # otherwise, just use the initial response
    else:
        response = lm["initial_response"]
        
    return True, response


guidance_agent = AssistantAgent("guidance_coder", llm_config=llm_config)
guidance_agent.register_reply(Agent, generate_structured_response, 1)
user_proxy = UserProxyAgent("user", human_input_mode="TERMINATE", code_execution_config={"work_dir": "coding"},
                            is_termination_msg=lambda msg: "TERMINATE" in msg.get("content"))
user_proxy.initiate_chat(guidance_agent, message="Plot and save a chart of nvidia and tsla stock price change YTD.")

user (to guidance_coder):

Plot and save a chart of nvidia and tsla stock price change YTD.

--------------------------------------------------------------------------------
guidance_coder (to user):

```python
# filename: stock_price_change.py

import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

# Get today's date
today = datetime.today().strftime('%Y-%m-%d')

# Download stock data
nvda = yf.download('NVDA', start='2022-01-01', end=today)
tsla = yf.download('TSLA', start='2022-01-01', end=today)

# Calculate percentage change in closing price
nvda['Pct Change'] = nvda['Close'].pct_change() * 100
tsla['Pct Change'] = tsla['Close'].pct_change() * 100

# Plot percentage change
plt.figure(figsize=(14, 7))
plt.plot(nvda['Pct Change'], label='NVDA')
plt.plot(tsla['Pct Change'], label='TSLA')
plt.title('Nvidia and Tesla Stock Price Change YTD')
plt.xlabel('Date')
plt.ylabel('Percentage Change (%)')
plt.legend()
plt.grid(True)

# Save the p

execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change


user (to guidance_coder):

exitcode: 0 (execution succeeded)
Code output: 

[*********************100%%**********************]  1 of 1 completed

[*********************100%%**********************]  1 of 1 completed


--------------------------------------------------------------------------------
guidance_coder (to user):

Great! The code executed successfully and the chart has been saved as 'stock_price_change.png' in the current directory. The chart shows the percentage change in the closing prices of Nvidia (NVDA) and Tesla (TSLA) stocks year-to-date. 

Please check the current directory for the 'stock_price_change.png' file to view the chart.

TERMINATE

--------------------------------------------------------------------------------
